__Author: Christian Camilo Urcuqui__

__Date:  18 January 2018__


![image](../../Utilities/genetic-algorithms.jpg)

This notebook is a compilation of different ideas that I take from the references in order to make a brief of genetic algorithms and its application with Python.

# Introduction to genetic algorithms

Genetic algorithms are one of the tools we can use to apply machine learning to finding good, sometimes even optimal, solutions to problems that have billions of potential solutions. It's biological approach is applied in software to find answers to problems that have really large search spaces by continuously generating cantidate solutions, evaluating how well the solutions fit the desired outcome, and refining the best solutions.

Genetic algorithms use random exploration of the problem space combined with evolutionary processes like mutation and
crossover (exchange of genetic information) to improve guesses. But also, because they have no experience in the problem
domain, they try things a human would never think to try. Thus, a person using a genetic algorithm may learn more about
the problem space and potential solutions. This gives them the ability to make improvements to the algorithm, in a
virtuous cycle.

__The genetic algorithm should make informed guesses__


## pseudo code

```
Function genetic_algorithm():
    it generates a initial random poblation
    end = False
    
    while is not end:
        for size(poblation)/2:
        select two individuals
        cross two individuals
        mutate with a probability the new individuals 
        add the new individuals to the poblation
        eliminate the two worst individuals from the poblation
        
   if is the poblation converges
       end = True
       
   exit with the best solution
        
```

## Guess the Password

Start with a randomly generated initial sequence of letters, then mutate/change one random letter at a time until the sequence of letters is "Hello World!"

In [1]:
import random

# genes
geneSet = " abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!."
target = "Hello World!"

In [2]:
# --generate a guess--
# the algorithm needs a way to generate a random string from the gene set
def generate_parent(length):
    genes = []
    while len(genes) < length:
        sampleSize =  min(length - len(genes), len(geneSet))
        # random.sample takes sampleSize values from the input without replacement
        genes.extend(random.sample(geneSet, sampleSize))
    return ''.join(genes)

generate_parent(3)

'JlQ'

In [13]:
# --fitness--
# the fitness value the genetic algorithm provides is the only feedback the engine gets to guide it toward a solution.
def get_fitness(guess):
    return sum(1 for expected, actual in zip(target, guess)
              if expected == actual)

get_fitness("z!")
#get_fitness("Hello World!")
#len(target)

0

In [14]:
#--mutate--
# the engine needs a way to produce a new guess by mutating the current one. The following implementation converts the parent 
# string to an array with list(parent), then replaces 1 letter in the array with a randomly selected one from geneSet, 
# and finally recombines the result into a string with ''.join(childGenes)
def mutate(parent):
    index =  random.randrange(0, len(parent))
    childGenes =  list(parent)
    newGene, alternate = random.sample(geneSet, 2)
    childGenes[index] = alternate if newGene == childGenes[index] else newGene
    return ''.join(childGenes)

mutate("hll")

'hlo'

In [21]:
# displaying 

import datetime

def display(guess):
    timeDiff = datetime.datetime.now() - startTime
    fitness =  get_fitness(guess)
    print("{0}\t{1}\t{2}".format(guess, fitness, str(timeDiff)))

In [23]:
random.seed()
startTime = datetime.datetime.now()
bestParent = generate_parent(len(target))
bestFitness = get_fitness(bestParent)
display(bestParent)
while True:
    child = mutate(bestParent)
    childFitness = get_fitness(child)
    if bestFitness >= childFitness:
        continue
    display(child)
    if childFitness >= len(bestParent):
        break
    bestFitness = childFitness
    bestParent = child

yNJcodakEVzt	1	0:00:00
HNJcodakEVzt	2	0:00:00
HNJco akEVzt	3	0:00:00.002028
HNJco akrVzt	4	0:00:00.002028
HNJco WkrVzt	5	0:00:00.002028
HNlco WkrVzt	6	0:00:00.003031
HNllo WkrVzt	7	0:00:00.003031
HNllo Wkrlzt	8	0:00:00.005036
Hello Wkrlzt	9	0:00:00.010025
Hello Wkrldt	10	0:00:00.012030
Hello Worldt	11	0:00:00.016058
Hello World!	12	0:00:00.016058


## Extract a resuable engine

It is time to modify the previous code to use it in future work, in the next steps we are going to change some methods, such as, generate_parent and mutate in order to receive any geneSet.

We are going to make it in a new file _genetic.py_

In [1]:
import random

def generate_parent(length, geneSet):
    genes=[]
    while len(genes) < length:
        sampleSize = min(length -  len(genes), len(geneSet))
        genes.extend(random.sample(geneSet, sampleSize))
    return ''.join(genes)

def mutate(parent, geneSet):
    index = random.randrange(0, len(parent))
    childGenes = list(parent)
    newGene, alternate = random.sample(geneSet, 2)
    childGenes[index] = alternate if newGene == childGenes[index] else newGene
    return ''.join(childGenes)

The next step is to move the main loop into a new public function named get_best in the genetic module. It's parameters are:

+ it requests the fitness for a guess
+ the number of genes to use when creating a new gene sequence
+ the optimal fitness value
+ the set of genes to use for creating and mutating gene sequences
+ the function to display, or report each improvement found

In [2]:
def get_best(get_fitness, targetLen, optimalFitness, geneSet, display):
    random.seed()
    bestParent = generate_parent(targetLen, geneSet)
    bestFitness = get_fitness(bestParent)
    display(bestParent)
    if bestFitness >= optimalFitness:
        return bestParent
    while True:
        child = mutate(bestParent, geneSet)
        childFitness =  get_fitness(child)
        
        if bestFitness >= childFitness:
            continue
        display(child)
        if childFitness >= optimalFitness:
            return child
        bestFitness = childFitness
        bestParent =  child

### Use the genetic module

In [12]:
import datetime 
import genetic

In [21]:
def test_Hello_World():
    target = "Hello World!"
    guess_password(target)

def guess_password(target):
    geneset = " abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!."
    startTime = datetime.datetime.now()
    
    def fnGetFitness(genes):
        return get_fitness(genes, target)

    def fnDisplay(genes):
        display(genes, target, startTime)

    optimalFitness = len(target)
    genetic.get_best(fnGetFitness, len(target), optimalFitness, geneset, fnDisplay)
    
def display(genes, target, startTime):
    timeDiff = datetime.datetime.now() - startTime
    fitness = get_fitness(genes, target)
    print("{0}\t{1}\t{2}".format(genes, fitness, str(timeDiff)))
    
def get_fitness(genes, target):
    return sum(1 for expected, actual in zip(target, genes)
        if expected == actual)

In [22]:
test_Hello_World()

dAscanuBzIHw	0	0:00:00.001003
dAscanuozIHw	1	0:00:00.001003
HAscanuozIHw	2	0:00:00.001986
HAlcanuozIHw	3	0:00:00.001986
HAllanuozIHw	4	0:00:00.001986
HAllanuozIH!	5	0:00:00.002989
HAllonuozIH!	6	0:00:00.002989
HAllonWozIH!	7	0:00:00.004019
HAllo WozIH!	8	0:00:00.004019
Hello WozIH!	9	0:00:00.004019
Hello WozId!	10	0:00:00.006023
Hello WorId!	11	0:00:00.008028
Hello World!	12	0:00:00.008028


## Use Python’s unittest framework

To do that the main test function must be moved into a class that inherits from unittest.TestCase. Let's make the new class and use the unittest in order to know the performance of our genetic algorithm.

You can call the algorithm using the next command

`
python -m unittest -b guessPasswordTests.py 
`

### another hard password

In [26]:
def test_For_I_am_fearfully_and_wonderfully_made():
    target = "For I am fearfully and wonderfully made."
    guess_password(target)

def guess_password(target):
    geneset = " abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!."
    startTime = datetime.datetime.now()
    
    def fnGetFitness(genes):
        return get_fitness(genes, target)

    def fnDisplay(genes):
        display(genes, target, startTime)

    optimalFitness = len(target)
    genetic.get_best(fnGetFitness, len(target), optimalFitness, geneset, fnDisplay)
    
def display(genes, target, startTime):
    timeDiff = datetime.datetime.now() - startTime
    fitness = get_fitness(genes, target)
    print("{0}\t{1}\t{2}".format(genes, fitness, str(timeDiff)))
    
def get_fitness(genes, target):
    return sum(1 for expected, actual in zip(target, genes)
        if expected == actual)

test_For_I_am_fearfully_and_wonderfully_made()

kRaJTYMgpbCAlcdShs eKXfBPqDNrLWQ!vFyEzGI	2	0:00:00.001004
kRaJTYMg bCAlcdShs eKXfBPqDNrLWQ!vFyEzGI	3	0:00:00.001004
kRaJTYMg fCAlcdShs eKXfBPqDNrLWQ!vFyEzGI	4	0:00:00.001004
kRaJTYMg fCAlfdShs eKXfBPqDNrLWQ!vFyEzGI	5	0:00:00.001004
kRaJTYMg fCAlfdShs eKXfBPnDNrLWQ!vFyEzGI	6	0:00:00.002006
kRaJTYMg fCAlfdShs eKXfBPnDerLWQ!vFyEzGI	7	0:00:00.002999
kRrJTYMg fCAlfdShs eKXfBPnDerLWQ!vFyEzGI	8	0:00:00.002999
kRrJTYMg fCAlfdShs eKXfBPnDerLWQ!vFyEzeI	9	0:00:00.002999
kRrJTYMg fCAlfdShs eKXfwPnDerLWQ!vFyEzeI	10	0:00:00.002999
kRrJTYMg fCAlfdShy eKXfwPnDerLWQ!vFyEzeI	11	0:00:00.002999
kRrJIYMg fCAlfdShy eKXfwPnDerLWQ!vFyEzeI	12	0:00:00.003992
kRrJIYMg fCAlfdlhy eKXfwPnDerLWQ!vFyEzeI	13	0:00:00.005022
kRrJIYag fCAlfdlhy eKXfwPnDerLWQ!vFyEzeI	14	0:00:00.008029
korJIYag fCAlfdlhy eKXfwPnDerLWQ!vFyEzeI	15	0:00:00.009033
korJIYag fCAlfdlhy eKXfwPnderLWQ!vFyEzeI	16	0:00:00.009033
korJIYag fCAlfdlly eKXfwPnderLWQ!vFyEzeI	17	0:00:00.010037
korJIYam fCAlfdlly eKXfwPnderLWQ!vFyEzeI	18	0:00:00.011040
korJI

## Introduce a Chromosome object

The next change is to introduce a Chromosome object that has Genes and Fitness attributes. This will make the genetic engine more flexible by making it possible to pass those values around as a unit. 

The next code is going to be in the _genetic2.py_.

In [1]:
class Chromosome:
    Genes = None
    Fitness =  None
    
    def __init__(self, genes, fitness):
        self.Genes = genes
        self.Fitness = fitness
    
    def mutate(parent, geneSet, get_fitness):
        index = random.randrange(0, len(parent.Genes))
        childGenes = list(parent.Genes)
        newGene, alternate = random.sample(geneSet, 2)
        childGenes[index] = alternate if newGene == childGenes[index] else newGene
        genes = ''.join(childGenes)
        fitness = get_fitness(genes)
        return Chromosome(genes, fitness)
    
    def generate_parent(length, geneSet, get_fitness):
        genes=[]
        while len(genes) < length:
            sampleSize = min(length -  len(genes), len(geneSet))
            genes.extend(random.sample(geneSet, sampleSize))
        genes = ''.join(genes)
        fitness = get_fitness(genes)
        return Chromosome(genes, fitness)
    
    def get_best(get_fitness, targetLen, optimalFitness, geneSet, display):
        random.seed()
        bestParent = generate_parent(targetLen, geneSet, get_fitness)
        display(bestParent)
        if bestParent.Fitness >= optimalFitness:
            return bestParent
        while True:
            child = mutate(bestParent, geneSet, get_fitness)
            

            if bestParent.Fitness >= child.Fitness:
                continue
            display(child)
            if child.Fitness >= optimalFitness:
                return child            
            bestParent = child

We need to change to the algorithm file functions in order to reduce some double work, let's make them in a new file _GuessPasswordTests2.py_

## Benchmark

The next improvement is to add support for benchmarking to genetic because it is useful to know how long the engine
takes to find a solution on average and the standard deviation.

The implementation is going to be in the file _genetic2.py_


In [2]:
class Benchmark:
    @staticmethod
    def run(function):
        timings = []
        stdout = sys.stdout
        for i in range(100):
            sys.stdout = None
            startTime = time.time()
            function()
            seconds = time.time() - startTime
            sys.stdout = stdout
            timings.append(seconds)
            mean = sc.mean(timings)
            if i < 10 or i % 10 == 9:
                print("{0} {1:3.2f} {2:3.2f}".format(
                    1 + i, mean,
                    sc.std(timings)
                    if i > 1 else 0))


## References 

+ Sheppard, C. (2016). Genetic algorithms with python. Clinton Sheppard.